# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [53]:
# Libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [54]:
# your code here
data=pd.read_csv('C:/Users/TDias/Documents/GitHub/lab-advanced-pandas/your-code/data/Admission_predict.csv')
data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [55]:
# your code here
data.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [56]:
data.rename(columns={'LOR ':'LOR','Chance of Admit ':'Chance of Admit'},inplace=True)
data.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [57]:
# your code here
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [58]:
# your code here
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [59]:
# your code here
data.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [60]:
# your code here
data.groupby(['GRE Score','CGPA']).size()

GRE Score  CGPA
290        7.46    1
           7.56    1
293        7.80    1
294        7.36    1
           7.64    1
                  ..
340        9.60    1
           9.66    1
           9.74    1
           9.91    1
           9.92    1
Length: 385, dtype: int64

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [61]:
data.set_index(['GRE Score','CGPA'],inplace=True)
data

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...
324,9.04,381,110,3,3.5,3.5,1,0.82
325,9.11,382,107,3,3.0,3.5,1,0.84
330,9.45,383,116,4,5.0,4.5,1,0.91


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [62]:
# your code here
data.reset_index(inplace=True)
data

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91
383,312,8.78,384,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [69]:
# your code here
data1=data[(data.CGPA>9) & (data.Research==1)]
data1

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
4,330,9.34,5,115,5,4.5,3.0,1,0.90
10,328,9.10,11,112,4,4.0,4.5,1,0.78
19,328,9.50,20,116,5,5.0,5.0,1,0.94
20,334,9.70,21,119,5,5.0,4.5,1,0.95
...,...,...,...,...,...,...,...,...,...
379,329,9.23,380,111,4,4.5,4.0,1,0.89
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [73]:
# your code here
data2=data[(data.CGPA>9) & (data.SOP<3.5)]
data2['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [76]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    mean=col.mean()
    std=col.std()
    new_col=(col-mean)/std
    return new_col

standardize(data['CGPA'])

0      1.747900
1     -0.991211
2      0.121034
3     -0.642597
4      1.233279
         ...   
380    0.735259
381    0.851463
382    1.415886
383    0.303641
384    1.764500
Name: CGPA, Length: 385, dtype: float64

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [83]:
# your code here
data['CGPA_std']=standardize(data['CGPA'])
data['GRE_std']=standardize(data['GRE Score'])
data['LOR_std']=standardize(data['LOR'])
data

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713
...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.735259,0.628727,0.076740
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.851463,0.715239,0.076740
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.415886,1.147798,1.191647
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.303641,-0.409414,0.634193


We will generate the decision choice at random using the code below. Please run the cell.

In [84]:
# Libraries
from random import choices

In [88]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=data.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [90]:
# your code here
data['deciding_column']=data.lookup(data.index,decision_choice)
data

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.747900
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,-0.480713
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.735259,0.628727,0.076740,0.628727
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.851463,0.715239,0.076740,0.851463
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.415886,1.147798,1.191647,1.415886
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.303641,-0.409414,0.634193,-0.409414


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [92]:
# your code here
data['decision']=[1 if element>0.8 else 0 for element in data['deciding_column']]
data

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.747900,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,-0.480713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.735259,0.628727,0.076740,0.628727,0
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.851463,0.715239,0.076740,0.851463,1
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.415886,1.147798,1.191647,1.415886,1
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.303641,-0.409414,0.634193,-0.409414,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [93]:
# your code here
data['decision'].value_counts()

0    295
1     90
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [97]:
# your code here
data.columns = data.columns.str.strip('. ').str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
data

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.747900,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,-0.480713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.735259,0.628727,0.076740,0.628727,0
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.851463,0.715239,0.076740,0.851463,1
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.415886,1.147798,1.191647,1.415886,1
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.303641,-0.409414,0.634193,-0.409414,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [114]:
# your code here
data['adjusted_gre']=pd.cut([j+10 if i>=4 else j for i,j in zip(data['university_rating'],data['gre_score'])],4)
data.tail(50)

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
335,318,8.27,336,107,3,3.0,3.5,1,0.74,-0.542993,0.109657,0.076740,0.076740,0,"(305.0, 320.0]"
336,325,8.67,337,110,4,3.5,4.0,1,0.73,0.121034,0.715239,0.634193,0.715239,0,"(320.0, 335.0]"
337,303,8.06,338,100,2,3.0,3.5,1,0.64,-0.891607,-1.188021,0.076740,-0.891607,0,"(289.94, 305.0]"
338,300,8.17,339,102,3,3.5,2.5,0,0.63,-0.709000,-1.447556,-1.038166,-0.709000,0,"(289.94, 305.0]"
339,297,7.67,340,98,2,2.5,3.0,0,0.59,-1.539033,-1.707092,-0.480713,-0.480713,0,"(289.94, 305.0]"
340,317,8.12,341,106,2,2.0,3.5,0,0.73,-0.792003,0.023145,0.076740,-0.792003,0,"(305.0, 320.0]"
341,327,8.77,342,109,3,3.5,4.0,1,0.79,0.287041,0.888263,0.634193,0.287041,0,"(320.0, 335.0]"
342,301,7.89,343,104,2,3.5,3.5,1,0.68,-1.173818,-1.361044,0.076740,-1.173818,0,"(289.94, 305.0]"
343,314,7.64,344,105,2,2.5,2.0,0,0.70,-1.588835,-0.236391,-1.595619,-1.595619,0,"(305.0, 320.0]"
344,321,8.44,345,107,2,2.0,1.5,0,0.81,-0.260781,0.369192,-2.153072,0.369192,0,"(320.0, 335.0]"
